In [ ]:
#Install Packages
# !pip install faiss-cpu
# !pip install sentence-transformers

In [1]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:

df = pd.read_csv("sample_text.csv")
df.shape


(8, 2)

In [3]:
df.head()

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event


Step 1 : Create source embeddings for the text column

In [4]:
from sentence_transformers import SentenceTransformer

f:\langchain_project\stock_articles_qa_chatbot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 67.0kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 7.01kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 674kB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<?, ?B/s] 
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 219kB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:43<00:00, 9.97MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 14.2kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 830kB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<?, ?B/s] 
train_script.py: 100%|██████████| 13.1k/13.1k [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 600kB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 22.3kB/s]


In [6]:
vectors.shape

(8, 768)

In [7]:
dim = vectors.shape[1]
dim

768

Step 2 : Build a FAISS Index for vectors

In [8]:
import faiss

index = faiss.IndexFlatL2(dim)

Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [9]:
index.add(vectors)

index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000199E66E0F30> >

Step 4 : Encode search text using same encorder and normalize the output vector

In [10]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [11]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

Step 5: Search for similar vector in the FAISS index created

In [13]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844836, 1.4039094]], dtype=float32)

In [14]:
I.tolist()
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [15]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [16]:
search_query

'I want to buy a polo t-shirt'

We can see that using semantic search we can get the top similar sentences from the Faiss vector db.